In [210]:
%run ../utils.ipynb

In [211]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

import logging

In [212]:
plt.set_loglevel('WARNING')

In [213]:
mlogger = logging.getLogger('matplotlib')
mlogger.setLevel(logging.WARNING)

In [214]:
masks = {
        '2015': df['season'] == 2015,
        '2016': df['season'] == 2016,
        '2017': df['season'] == 2017,
        '2018': df['season'] == 2018,
        '2019': df['season'] == 2019,
        '2020': df['season'] == 2020,
        '2021': df['season'] == 2021,
        '2022': df['season'] == 2022,
        '2023': df['season'] == 2023
        }

In [215]:
data_dir = f"{project_dir}\\data\\matchup_player_data\\"

In [216]:
df = pd.read_csv(data_dir+'matchup_player_data_through_2023.csv')

In [217]:
df.shape

(2058, 58)

In [218]:
df.columns

Index(['QB', 'QB_player_key', 'QB_points', 'WR1', 'WR1_player_key',
       'WR1_points', 'WR2', 'WR2_player_key', 'WR2_points', 'RB1',
       'RB1_player_key', 'RB1_points', 'RB2', 'RB2_player_key', 'RB2_points',
       'TE', 'TE_player_key', 'TE_points', 'W/R/T', 'W/R/T_player_key',
       'W/R/T_points', 'BN1', 'BN1_player_key', 'BN1_points', 'BN2',
       'BN2_player_key', 'BN2_points', 'BN3', 'BN3_player_key', 'BN3_points',
       'BN4', 'BN4_player_key', 'BN4_points', 'BN5', 'BN5_player_key',
       'BN5_points', 'K', 'K_player_key', 'K_points', 'DEF', 'DEF_player_key',
       'DEF_points', 'BN6', 'BN6_points', 'team_name', 'manager', 'week',
       'BN6_player_key', 'season', 'BN7', 'BN7_player_key', 'BN7_points',
       'WR3', 'WR3_player_key', 'WR3_points', 'IR', 'IR_player_key',
       'IR_points'],
      dtype='object')

In [225]:
df['season'] = pd.to_datetime(df['season'], format="%Y")

In [219]:
position_columns = ['QB',
                   'WR1',
                   'WR2',
                   'WR3',
                   'RB1',
                   'RB2',
                   'TE',
                   'W/R/T'
                   ]
points_columns = [ x + '_points' for x in position_columns]    

In [220]:
df.loc[df['QB_points'] == df['QB_points'].max(), ['QB', 'QB_points', 'week','season']]

,QB,QB_points,week,season
987,Aaron Rodgers,43.76,7,2019


In [221]:
season_dfs = { k : df[v] for k,v in masks.items()}

In [233]:
@interact

def position_over_time_plt( pos = position_columns ):
    points_col = pos+'_points'
    filtered_df = df.loc[ :, [pos, 'season', points_col]]
    
    plt.figure(figsize=(10,10))
    sns.boxplot(data=df, x = 'season', y = points_col, hue= 'season', legend=False);
    

interactive(children=(Dropdown(description='pos', options=('QB', 'WR1', 'WR2', 'WR3', 'RB1', 'RB2', 'TE', 'W/R…

In [223]:
df.groupby('manager')['QB_points'].describe()

,count,mean,std,min,25%,50%,75%,max
manager,,,,,,,,
Big Rick,16.0,16.038125,7.222801,-0.03,13.4675,18.815,19.9025,27.77
Buddy,147.0,18.015238,8.461334,0.00,11.8700,17.280,22.9850,43.76
Eric,147.0,15.719320,9.417570,-7.83,10.6100,15.180,21.6900,41.82
Frags,147.0,16.645918,7.761891,-0.23,11.7500,16.370,21.4400,38.10
Jake,147.0,17.012109,8.252643,-0.48,10.9500,17.100,22.7050,40.93
Jones,147.0,18.520272,8.469762,0.00,12.6900,18.020,24.1250,41.88
Kyle,147.0,16.890272,7.771029,-3.00,11.1000,16.540,21.5900,40.06
Lauren,147.0,16.669932,8.271044,0.00,11.0700,16.100,22.4250,39.88
Michael,147.0,16.948027,7.810023,-1.30,12.0950,16.670,22.1050,37.92


In [226]:
@interact

def position_points_explorer(manager = df['manager'].unique(), pos = position_columns, season = df['season'].unique()):
    """
    Paints an interactive lollipop chart for given weekly player data allowing for selections by manager, position, and season
    
    """
    # filter the dataframe by selected manager and season, with values as selected position and points
    # for example, this df might look like: manager Michael, season 2018 columns QB & QB_points 
    filtered_df = df[(df['manager'] == manager) & (df['season'] == season)][[pos, pos+'_points']]
    filtered_df.reset_index(inplace=True)

    # average for the season for that position, used as reference horizontal line in chart
    league_average = df[df['season'] == season][pos+'_points'].mean()
    
    # RGB color array, sufficiently different from eachother to make reading easier
    label_colors = [(0.0, 0.286, 0.572),
                    (0.0, 0.619, 0.450),
                    (0.6, 0.8, 0.196),
                    (1.0, 0.6, 0.6),
                    (1.0, 0.8, 0.0),
                    (1.0, 0.6, 0.0),
                    (0.901, 0.098, 0.294),
                    (0.6, 0.0, 0.6),
                    (1.0, 0.4, 1.0),
                    (1.0, 0.8, 0.6),
                    (0.6, 0.4, 0.2),
                    (0.588, 0.588, 0.588),
                    (0.4, 0.6, 0.8),
                    (0.6, 0.8, 1.0),
                    (0.8, 0.6, 1.0)]
    
    # establish weeks to serve as x axis 
    weeks = range(1, len(filtered_df.index) + 1)

    # result would indicate the df is empty - meaning the manager didn't appear in the selected season, 
    if not len(filtered_df.index):
        return "It appears the selected manager did not play in this season!"
        
    # establish figure and draw horizontal line representing season average
    plt.figure(figsize=(10,8))
    plt.hlines(league_average, 0, weeks[-1],linestyles='dotted', color='r', alpha = 0.5, label = f'{season} League Average')

    # loop responsible for charting individual scatter points and lines
    
    color_index = 0
    for player in filtered_df[pos].unique():
        # find the weeks for each player that filled the position over the course of the season played 
        weeks_played = filtered_df[filtered_df[pos] == player].index

        # chart only points and lines relevant to the player in this loop 
        plt.scatter(x = weeks_played, y = filtered_df[filtered_df[pos] == player][pos+'_points'], label = player, \
                    color=label_colors[color_index], s=75)
        # chart corresponding vertical lines (the lollipop "stick") using league average as reference point
        plt.vlines(x = weeks_played,
                   ymin = league_average,
                   ymax= filtered_df[filtered_df[pos] == player][pos+'_points'],
                  colors = label_colors[color_index])
        color_index+=1

    # additional figure elements
    plt.grid(visible=True, axis='y', alpha=0.3)
    plt.title(f"{pos} position for {manager}, {season} Season")
    plt.xlabel('Week', size=12)
    plt.ylabel('Points Scored')
    plt.ylim(-10, 50)
    plt.legend()
    plt.show()

interactive(children=(Dropdown(description='manager', options=('Ricky', 'Peter', 'Tim', 'Big Rick', 'Michael',…